In [1]:
import pandas as pd
 
df = pd.read_csv("cortes_luz.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.iloc[11:31, :]

In [2]:
def extractDate(string):
  nums_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
  for s in string:
    if s in nums_list:
      idx = string.find(s)
      break
  return string[idx:].replace(' de', '').lower()

def convMonth(date):
  dict_months = {"enero":"1", "febrero":"2", "marzo":"3", "abril":"4", "mayo":"5", "junio":"6", 
                "julio":"7", "agosto":"8", "septiembre":"9", "octubre":"10", "noviembre":"11", "diciembre":"12"}
  # "1/enero/2021" ->['1', 'enero', '2021'] 
  date_split = date.split(sep="/")
  date_split[1] = dict_months[date_split[1]]
  return "/".join(date_split)

df['fechas'] = df['fechas'].apply( lambda x: extractDate(x).replace(" ", "/") if x.split()[-1] in ['2020', '2021', '2022'] else  (extractDate(x)+ ' 2020').replace(" ", "/") )
df['fechas'] = df['fechas'].apply( lambda x:  convMonth(x))
df.head(5)

,fechas,datos
11,31/8/2022,Por ajustes en el servicio Enel Colombia reali...
12,30/8/2022,Por ajustes en el servicio Enel Colombia reali...
13,29/8/2022,Por ajustes en el servicio Enel Colombia reali...
14,26/8/2022,Por ajustes en el servicio Enel Colombia reali...
15,25/8/2022,Por ajustes en el servicio Enel Colombia reali...


In [3]:
allText = []

for i in range(len(df)):
  loc_aux = ""
  for j in df.iloc[i,1].lower().split('\n'):
    characters = "·•-.-" 
    j = ''.join( x for x in j if x not in characters) # string sin símbolos
    if j.startswith('localidad'):
      loc_aux = j

    elif j.startswith('desde'):
      k = j + "{}fecha{}".format(loc_aux, df.iloc[i, 0])
      allText.append(k)

In [4]:
fixLists = []

for i in allText:

  i = i.replace("hrs", "//")
  i = i.replace("dela", "//")
  
  i = i.replace("localidad", "//LOCALIDAD")
  i = i.replace("barrio", "//BARRIO")
  i = i.replace("fecha", "//FECHA")
  
  fixLists.append(i.split(sep = "//"))

In [5]:
new_df = pd.DataFrame(fixLists)
new_df = new_df.drop([6],axis=1)
new_df.columns = ['hora inicio', 'hora fin', 'direccion', 'barrio', 'localidad', 'fecha']
new_df.head(5)

,hora inicio,hora fin,direccion,barrio,localidad,fecha
0,desde las 07:45,hasta las 16:15,de la cr 85 a cr87 entre cl 52 sur a cl54 sur,BARRIO chicalá,LOCALIDAD bosa,FECHA31/8/2022
1,desde las 08:00,hasta las 17:00,de la cl66 sur a cl69 sur entre cr 17 a cr 20,BARRIO compartir,LOCALIDAD ciudad bolívar,FECHA31/8/2022
2,desde las 07:00,hasta las 08:30,de la cr14 a cr 16 entre cl 37 a cl39,BARRIO la magdalena,LOCALIDAD teusaquillo,FECHA31/8/2022
3,desde las 07:00,hasta las 17:30,de la ci54 a ci58 entre cr 44 a cr 48,BARRIO nicolás de federmán,LOCALIDAD teusaquillo,FECHA31/8/2022
4,desde las 08:00,hasta las 17:00,de la cr16 a cr19 entre cl 50 a cl52,BARRIO alfonso lópez,LOCALIDAD teusaquillo,FECHA31/8/2022


In [6]:
def extractHours(string):
    str_aux = ""
    for s in string:
        if s.isnumeric():
            str_aux += s
    return str_aux

def extractRestData(string):
    string = str(string)
    if string.startswith("BARRIO "):
        return string[len("BARRIO "):]
    elif string.startswith("LOCALIDAD "):
        return string[len("LOCALIDAD "):]
    elif string.startswith("FECHA"):
        return string[len("FECHA"):]

new_df['hora inicio'] = new_df['hora inicio'].apply(lambda x: extractHours(x))
new_df['hora fin'] = new_df['hora fin'].apply(lambda x: extractHours(x))
new_df['barrio'] = new_df['barrio'].apply(lambda x: extractRestData(x))
new_df['localidad'] = new_df['localidad'].apply(lambda x: extractRestData(x))
new_df['fecha'] = new_df['fecha'].apply(lambda x: extractRestData(x))
new_df

,hora inicio,hora fin,direccion,barrio,localidad,fecha
0,0745,1615,de la cr 85 a cr87 entre cl 52 sur a cl54 sur,chicalá,bosa,31/8/2022
1,0800,1700,de la cl66 sur a cl69 sur entre cr 17 a cr 20,compartir,ciudad bolívar,31/8/2022
2,0700,0830,de la cr14 a cr 16 entre cl 37 a cl39,la magdalena,teusaquillo,31/8/2022
3,0700,1730,de la ci54 a ci58 entre cr 44 a cr 48,nicolás de federmán,teusaquillo,31/8/2022
4,0800,1700,de la cr16 a cr19 entre cl 50 a cl52,alfonso lópez,teusaquillo,31/8/2022
...,...,...,...,...,...,...
364,1600,1730,de la ci 20 a ci22 entre cr 69 a cr 71,franco,fontibón,2/8/2022
365,1600,1730,de la cr68 a cr 70 entre cl22 a cl 24,terminal de transportes,fontibón,2/8/2022
366,1600,1730,de la cr19a cr21 entre cl 74 a cl76,san felipe,,None
367,0700,0830,de la cr16 a cr 18 entre cl 101 a cl 103,santa bibiana,usaquén,2/8/2022


In [7]:
def delSpaces(s):
  if type(s) == str:
    return s.lstrip().rstrip()
  else:
    pass
  
new_df.applymap(delSpaces)
new_df = new_df.dropna()

In [8]:
def postCode(df):
  from geocoder import osm
  if type(df[3]) == str and type(df[4]) == str:
    try:
      loc = osm('{} {} bogota colombia'.format(df[3], df[4]))
      return str(loc.raw['address']['postcode'])
    except:
      pass


In [9]:
new_df['post code'] = new_df.apply(postCode, axis=1)
new_df

C:\Users\santiago.1173\AppData\Local\Temp\ipykernel_14920\3894160068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['post code'] = new_df.apply(postCode, axis=1)


,hora inicio,hora fin,direccion,barrio,localidad,fecha,post code
0,0745,1615,de la cr 85 a cr87 entre cl 52 sur a cl54 sur,chicalá,bosa,31/8/2022,110711
1,0800,1700,de la cl66 sur a cl69 sur entre cr 17 a cr 20,compartir,ciudad bolívar,31/8/2022,111951
2,0700,0830,de la cr14 a cr 16 entre cl 37 a cl39,la magdalena,teusaquillo,31/8/2022,111311
3,0700,1730,de la ci54 a ci58 entre cr 44 a cr 48,nicolás de federmán,teusaquillo,31/8/2022,111321
4,0800,1700,de la cr16 a cr19 entre cl 50 a cl52,alfonso lópez,teusaquillo,31/8/2022,111311
...,...,...,...,...,...,...,...
363,1400,1530,de la ci 22 a cl 24 entre cr68 a cr 70,la esperanza sur,fontibón,2/8/2022,110931
364,1600,1730,de la ci 20 a ci22 entre cr 69 a cr 71,franco,fontibón,2/8/2022,110931
365,1600,1730,de la cr68 a cr 70 entre cl22 a cl 24,terminal de transportes,fontibón,2/8/2022,None
367,0700,0830,de la cr16 a cr 18 entre cl 101 a cl 103,santa bibiana,usaquén,2/8/2022,110111


In [10]:
new_df.dropna(inplace=True)

C:\Users\santiago.1173\AppData\Local\Temp\ipykernel_14920\1925939579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


In [11]:
new_df

,hora inicio,hora fin,direccion,barrio,localidad,fecha,post code
0,0745,1615,de la cr 85 a cr87 entre cl 52 sur a cl54 sur,chicalá,bosa,31/8/2022,110711
1,0800,1700,de la cl66 sur a cl69 sur entre cr 17 a cr 20,compartir,ciudad bolívar,31/8/2022,111951
2,0700,0830,de la cr14 a cr 16 entre cl 37 a cl39,la magdalena,teusaquillo,31/8/2022,111311
3,0700,1730,de la ci54 a ci58 entre cr 44 a cr 48,nicolás de federmán,teusaquillo,31/8/2022,111321
4,0800,1700,de la cr16 a cr19 entre cl 50 a cl52,alfonso lópez,teusaquillo,31/8/2022,111311
...,...,...,...,...,...,...,...
361,0900,1030,de la cl 0 a cl 2 entre cr 3 a cr 5,las cruces,santa fe,2/8/2022,110321
362,0900,1230,de la ci 15a cl 17 entre cr 101 a cr 105,el carmen fontibón,fontibón,2/8/2022,110921
363,1400,1530,de la ci 22 a cl 24 entre cr68 a cr 70,la esperanza sur,fontibón,2/8/2022,110931
364,1600,1730,de la ci 20 a ci22 entre cr 69 a cr 71,franco,fontibón,2/8/2022,110931


In [14]:
new_df.to_excel('cortesAlcaldia.xlsx')

In [12]:
import geocoder
loc = geocoder.osm('cl 14a #5a-23 bogota colombia')
print('Latitud: {}, Longitud: {}'.format(loc.latlng[0],loc.latlng[1]))

Latitud: 4.6504861, Longitud: -74.1340262


In [13]:
loc.raw['address']['postcode']

'110821'